In [8]:
import xml.etree.cElementTree as et
import pandas as pd
import numpy as np
import os
import xlrd

DIR_PATH = 'nano-ds'
TEST_DIR = 'testdata'

def getvalueofnode(node):
    return str(node.text) if node is not None else 'None'

def getEmptyIfNone(text):
    return text if text is not None else 'None'

In [13]:

def getXML2Text(rootDir):
    
    dfcols = ['doi','title','abstract','keywords','body']
    df = pd.DataFrame(columns=dfcols)
    typeDirs = [TEST_DIR]
   
    for typeDir in typeDirs:
        for root, dirs, files in os.walk(rootDir + '/' + typeDir):
            for filename in files:
 
                #Todo: Remove it later
                #print(filename)
                if not filename.startswith('.'):
                    #print(rootDir + '/' + typeDir + '/' + filename)
                    parsed_xml = et.parse(rootDir + '/' + typeDir + '/' + filename)
                    for node in parsed_xml.getroot():
                        if node.tag == 'Journal':
                            doi = node.find('Volume/Issue/Article/ArticleInfo/ArticleDOI')
                            titleElem = node.find('Volume/Issue/Article/ArticleInfo')
                            abstractElem = node.find('Volume/Issue/Article/ArticleHeader')
                            body = node.find('Volume/Issue/Article/Body')
                            keywordGroup = node.find('Volume/Issue/Article/ArticleHeader')
                            #print(body)
                            ## Keyword sometimes have special tags as child nodes
                            
                            keyword_list = []
                            bodyPageList = []
                            abstractValue = ""
                            title = ""
                            
                            if titleElem != None:
                                if titleElem.find('ArticleTitle') != None:
                                    for articleTitle in titleElem.iter("ArticleTitle"):
                                        for tags in articleTitle.itertext():
                                            title += tags.strip()
                                else:
                                    title = 'None'
                            else:
                                title = 'None'
            
                            
                            if abstractElem != None:
                                if abstractElem.find('Abstract') != None:
                                    for para in abstractElem.iter("Para"):
                                        for abstract in para.itertext():
                                            abstractValue += abstract.strip()
                                else:
                                    abstractValue = 'None'
                            else:
                                abstractValue = 'None'
                        
 
                            if keywordGroup != None:        
                                if keywordGroup.find('keywordGroup') != None:
                                    for keyword in keywordGroup.iter("Keyword"):
                                        keywords = ""
                                        for txt in keyword.itertext():
                                            keywords += txt.strip()
                                        keyword_list.append(keywords)          
                                else:
                                    keyword_list.append('None')
                                keyword_str = ','.join(map(str,keyword_list))
                            else:
                                keyword_list.append('None')

                            if body != None:
                                if body.find('page') != None:
                                    pageList = []
                                    for page in body.iter("page"):
                                        pageList.append(page)
                                    pageWithoutTags = ""
                                    for pageTags in pageList[-1].itertext():
                                        pageWithoutTags += pageTags.strip()
                                    bodyPageList.append(pageWithoutTags)
                                    bodyPage = " ".join(map(str,bodyPageList))
                                else:
                                    paraList = []
                                    for para in body.iter("Para"):                                        
                                        paraList.append(para)
                                        paraWithoutTags = ""
                                    for paraTags in pageList[-1].itertext():
                                        paraWithoutTags += paraTags.strip()
                                    bodyPageList.append(paraWithoutTags)
                                    bodyPage = " ".join(map(str,bodyPageList))
                            else:
                                bodyPageList.append('None')
                                bodyPage = " ".join(map(str,bodyPageList))

                            #bodyPage = " ".join(map(str,bodyPageList))
                            df = df.append(pd.Series([getvalueofnode(doi),title,abstractValue,keyword_str, bodyPage], index=dfcols), ignore_index=True)
                            #df['concepts'].apply(lambda x: x.decode("utf-8"))
    ddd=df.ix[:,0]
    listOfValues = df.values
    finalList = []
    strRowList = []
    for strRow in listOfValues:
        strRowList.append(strRow)
        strSectionsList = []
        innerString = ""
        for strSections in strRow:
            strSectionsList.append(strSections)
        innerString = " ".join(strSectionsList)
        finalList.append(innerString)
    #return finalList
    
    #df.to_csv("temp.csv",index=False)
    #np.savetxt(r'dataV8_lastBody.txt', df.values, fmt='%s')
    #np.savetxt(r"parsed.txt", df.values, fmt='%s')
    
    with open('doiList.txt', mode='wt', encoding='utf-8') as myfile:
        for lines in ddd:
            print(lines, file = myfile)
    myfile.close
    

In [14]:
def main():
    #getMapDoiConceptFromExcel(DIR_PATH)
    getXML2Text(DIR_PATH)
    print('Done...')
main()

Done...


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:93: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
